# NGC 7000 (North American Nebula) 3D Velocity Analysis

*Eric G. Suchanek, Ph.D. 8/26/19*

<img src ="https://www.constellation-guide.com/wp-content/uploads/2018/12/North-America-Nebula.jpg">

This code analyzes stellar data extracted from the Gaia DR2 repository via my iPython notebook: <a href="./ExtractStars_batch_all.ipynb">ExtractStars_batch_all.ipynb</a>. The data files are simple .csv files where the column headers are the Gaia data fields. In addition the VX, VY, VZ components and PM angle are included. Several interactive visualizations are performed to see distributions across the various velocity and distance variables. The extracts are calculated at differing levels of statistical significance, and that is reflected in the vel_filename variable.

<i>NB: the code relies on the routines stored in the file: <a href="./egs.py">egs.py</a> which should be located in the same directory as this notebook.</i>


In [ ]:
#
# Important libraries and global variables that the following cells rely on. Run this first. -egs-
#
# Suppress warnings. Comment this out if you wish to see the warning messages
import warnings
warnings.filterwarnings('ignore')

import plotly.offline
#import plotly.plotly as py
import cufflinks as cf
import ipyvolume as ipv

import pandas as pd
from pandas import DataFrame
from pandas import read_csv

import ipywidgets as widgets
from ipywidgets import interact, interact_manual
from ipywidgets import FloatSlider, ColorPicker, VBox, jslink
from time import time

import numpy as np

import egs

# the file egs.py must be included in the same directory as the notebook!
#from egs import myshrink, ra_dec_to_xyz
#from egs import pprint_delta, return_constraint
#from egs import normalize_Velocity, compute_3d_velocities_pandas_pc

# Global variables used by the following cells. If you export the cells to straight Python remember to
# include the appropriate libraries and variables from this area of the notebook!

# A string name for the object we're analyzing. Used in the graphs
obj_name = "NGC 7000 "

# the current stellar data file generated by ExtractStars.py. Make sure the file exists at the path
# specified!
#vel_filename = "./PM_all/NGC7000PM_20h58m47s_44d19m48s_1.0x1.0_all_vel.csv"
#vel_filename = "./PM_all/NGC7000PM_20h58m47s_44d19m48s_0.25x0.25_all_vel.csv"
vel_filename = "./PM_all/NGC7000PM_20h58m47s_44d19m48s_0.5x0.5_all_vel.csv"

_figwidth=800
_figheight=800

#
# We go offline with cufflinks first, since this is all local
#
cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)

start = time()
# read the data back into a data frame
df = egs.read_Gaia_stars(vel_filename)
if df is not False:
    df2 = DataFrame()
    df2["pmra"] = df["pmra"]
    df2["pmdec"] = df["pmdec"]
    df2["ra"] = df["ra"]
    df2["dec"] = df["dec"]
    df2["Distance"] = df["Distance"]
    df2["PM_Angles"] = df["PM_Angles"]
    df2["VX"] = df["VX"]
    df2["VY"] = df["VY"]
    df2["VZ"] = df["VZ"]
    df2["parallax_over_error"] = df["parallax_over_error"]
    star_count = len(df2)
    print('Read in:',star_count,'stars')    
else:
    print("Can't open file: ", vel_filename)

egs.pprint_elapsed(start)
print()

Plot an interactive histogram of count vs proper motion in RA and Dec 

In [ ]:
from jupyterthemes import jtplot

jtplot.style(theme='monokai')
start = time()
df2[["pmra","pmdec"]].iplot(kind="histogram", bins=200, theme='plotly_dark',
                title=str.format("{}Proper Motion, {} stars", obj_name, star_count),
                xTitle='Proper Motion (mas/yr)', yTitle='Count')

end = time()
egs.pprint_elapsed(start)
print()

Plot an interactive histogram of stellar count vs proper motion angles

In [ ]:
start = time()
df2["PM_Angles"].iplot(kind="histogram", bins=180,theme="white",
                title=str.format("{}Proper Motion Angles, {} stars",obj_name,star_count),
                xTitle='Proper Motion angle',yTitle='Count')

egs.pprint_elapsed(start)
print()

Plot an interactive histogram of count vs distance (pc)

In [ ]:
start = time()
df2["Distance"].iplot(kind="histogram", bins=200,theme="white",
                title=str.format("{}Distances, {} stars",obj_name,star_count),
                xTitle='Distance (pc)',yTitle='Count')

egs.pprint_elapsed(start)
print()

Plot a correlation plot

In [ ]:
df2.corr().iplot(kind='heatmap', colorscale="Blues",
                title="NGC7000 Correlations")

In [ ]:

start = time()

# interactive tabular query
@interact
def show_items_less_than(column=['Distance', 'VX', 'VY', 'VZ'], 
                            x=(0, 20000, 1)):
    return df2.loc[df2[column] < x]

egs.pprint_elapsed(start)
print()

Interactive ipyvol quiver plot of vx, vy, vz against distance. The arrow orientation corresponds to the individual star's velocity vectors VX, VY, VZ

In [ ]:
#ipywidgets-display::
# do a 3d quiver plot of VX, VY, VZ against distance 

start = time()

# return a new dataframe containing stars between low and high distances (pc)
df3 = DataFrame()
df3 = return_constraint(df2,'Distance',500,800)
# normalize the velocities
df3 = normalize_Velocity(df3)
cnt = len(df3)

# extract RA, Dec, Distance for the plot
X = np.asarray(df3['ra'])
Y = np.asarray(df3['dec'])
Z = np.asarray(df3['Distance'])

xmin,xmax,ymin,ymax,zmin,zmax = myshrink(X,Y,Z,0.0)

# extract the velocities
VX = df3['VX']
VY = df3['VY']
VZ = df3['VZ']

print(" Before distance cutoff there were:", star_count, "stars.", "\n", "After cutoff there were: ", cnt, "remaining")

# slice the VX, VY, VZ columns for input into the plot
VX = np.asarray(df3['VX'])
VY = np.asarray(df3['VY'])
VZ = np.asarray(df3['VZ'])

# set up the plot
ipv.clear()
ipv.figure(width=_figwidth,height=_figheight)
ipv.style.use("dark") # looks better
ipv.xyzlabel('RA','Dec', 'Dist (pc)')
ipv.xlim(xmin,xmax)
ipv.ylim(ymin,ymax)
ipv.zlim(zmin,zmax)

quiver = ipv.quiver(X,Y,Z,VX,VY,VZ,size=2,size_selected=6,alpha=.6)

# wire up a slider and color picker
sz = FloatSlider(min=0, max=30, step=0.1)
col = ColorPicker()
jslink((quiver, 'size'), (sz, 'value'))
jslink((quiver, 'color'), (col, 'value'))

# display the rendering
VBox([ipv.gcc(), sz, col])

egs.pprint_elapsed(start)
print()


In [ ]:
# interactive scatter plot
start = time()
@interact
def scatter_plot(x=list(df3.select_dtypes('number').columns), 
                 y=list(df3.select_dtypes('number').columns)[2:],
                 theme=list(cf.themes.THEMES.keys()), 
                 colorscale=list(cf.colors._scales_names.keys())):
    
    df3.iplot(kind='scatter', x=x, y=y, mode='markers', 
             xTitle=x.title(), yTitle=y.title(), size=2,
             #text='title',
            title=f'{y.title()} vs {x.title()}',
            theme=theme, colorscale=colorscale)

egs.pprint_elapsed(start)
print()